## 1. Import Relevant Libraries and Reading of Data

* 1a. Importing Relevant Libraries

In [142]:
# Import Required Libaries
import keras
import keras.backend as K
from keras.layers.core import Activation
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.layers import LSTM, Dense, Dropout, Masking, TimeDistributed

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn import preprocessing

from keras.layers import Bidirectional

import math 

* 1b. Reading in the Data:
<br><i> > Training Data
<br> > Testing Data
<br> > True Labels (RUL) for the Testing Data

In [143]:
# Reading in the data (txt files)

# Train Data
train_df = pd.read_csv("/content/train_FD001.txt", sep=" ", header=None)
# Test Data
test_df = pd.read_csv("/content/test_FD001.txt", sep=" ", header=None)
# True RUL Data for the Test Set
truth_df = pd.read_csv("/content/RUL_FD001.txt", sep=" ", header=None)

* 1c. We set a seed for Reproducibility of Results

In [144]:
np.random.seed(1234)  
PYTHONHASHSEED = 0

* 1d. We create a path to save the Deep Learning model output 

In [145]:
# define a path to save model
model_path = '/content/Output/regression_model.h5'

## 2. Data Preprocessing
- Drop the last 2 columns which consists of all null values
- Rename columns according to the Engine ID, Cycle, the 3 respective Operating Settings and the 21 Operating Sensors.
- Derive the RUL from the train data. This is because the RUL for the train data is not explicity provided.
- MinMax Normalisation to transform our features' values to a value between 0 and 1; 0 represents the minimum output value of the sensor value, while 1 represents the maximum output value of the sensor value.
- Clipping the upper limit of the RUL of aircrafts to mimic a more accurate degradation pattern of the aircraft engine with increasing usage.
- Transform the data into a form (3-Dimensional Form) that can be fed into the deep learning model. 

* 2a. Drop the last 2 columns for Training, Testing and True Labels Dataframes
> This is because the last 2 columns consists of all null values.

In [146]:
# Drop the last 2 columns for Train, Test and True RUL Dataframes
train_df.drop(train_df.columns[[26, 27]], axis=1, inplace=True)
test_df.drop(test_df.columns[[26, 27]], axis=1, inplace=True)
truth_df.drop(truth_df.columns[[1]], axis=1, inplace=True)

* 2b. Rename Columns in this form: 
<br><i>-> Engine ID
<br>-> 3 Operating Settings
<br>-> 21 Sensors

In [147]:
# Rename columns into readable forms, namely, by: 
# [Engine ID, Operational Settings, Sensors]


train_df.columns = ['id', 'cycle', 'setting1', 'setting2', 'setting3', 's1', 's2', 's3',
                     's4', 's5', 's6', 's7', 's8', 's9', 's10', 's11', 's12', 's13', 's14',
                     's15', 's16', 's17', 's18', 's19', 's20', 's21']
train_df = train_df.sort_values(['id','cycle'])


test_df.columns = ['id', 'cycle', 'setting1', 'setting2', 'setting3', 's1', 's2', 's3',
                     's4', 's5', 's6', 's7', 's8', 's9', 's10', 's11', 's12', 's13', 's14',
                     's15', 's16', 's17', 's18', 's19', 's20', 's21']


* 2c. Derive RUL for the Training Data
<br><i> This is because the RUL for the Training Data is not explicity provided.

In [148]:
# Derive the RUL for the Train Data
def extract_rul(data, factor = 0):

    # Get the total number of cycles for each unit, i.e. Each Engine ID
    rul = pd.DataFrame(data.groupby('id')['cycle'].max()).reset_index()
    rul.columns = ['id', 'max']

    # Merge the maximum cycle into the original dataframe
    data = data.merge(rul, on=['id'], how='left')

    # Actual calculation of RUL
    data['RUL'] = data['max'] - data['cycle']

    # Drop the 'max' column, which is now redundant
    data.drop(columns=['max'], axis = 1, inplace = True)
  
    return data[data['cycle'] > factor]

# Apply the function on the training data
train_df = extract_rul(train_df, factor = 0)

* 2d. Clipping the maximum RUL to aircraft engines for Training Data 
> This is to more accurately portray the degradation pattern of aircraft engines after a certain period of usage

In [149]:
# Clipping the maximum RUL to aircraft engines (for train data)
train_df['RUL'] = train_df['RUL'].clip(upper=125)

* 2e. MinMax Normalisation of Training Data
> Transforms our features' values to a value between 0 and 1; 0 represents the minimum output value of the sensor value, while 1 represents the maximum output value of the sensor value.

In [150]:
# MinMax normalization of Operational Settings and Sensor Values (from 0 to 1) for train set
train_df['cycle_norm'] = train_df['cycle']
cols_normalize = train_df.columns.difference(['id','cycle','RUL'])
min_max_scaler = preprocessing.MinMaxScaler()
norm_train_df = pd.DataFrame(min_max_scaler.fit_transform(train_df[cols_normalize]), 
                             columns=cols_normalize, 
                             index=train_df.index)
join_df = train_df[train_df.columns.difference(cols_normalize)].join(norm_train_df)
train_df = join_df.reindex(columns = train_df.columns)

# MinMax normalization of Operational Settings and Sensor Values (from 0 to 1) for test set
test_df['cycle_norm'] = test_df['cycle']
norm_test_df = pd.DataFrame(min_max_scaler.transform(test_df[cols_normalize]), 
                            columns=cols_normalize, 
                            index=test_df.index)
test_join_df = test_df[test_df.columns.difference(cols_normalize)].join(norm_test_df)
test_df = test_join_df.reindex(columns = test_df.columns)
test_df = test_df.reset_index(drop=True)





* 2f. Processing of Test Data with the True RUL Labels of the Test Data

In [151]:
# Generate labels (the RUL) for the Test Data using the dataset containing the true RUL for the Test Data.
rul = pd.DataFrame(test_df.groupby('id')['cycle'].max()).reset_index()
rul.columns = ['id', 'max']
# rename column in Truth Dataframe
truth_df.columns = ['rul_init']
# assign the RUL to the Engine IDs by adding a column in Truth Dataframe
truth_df['id'] = truth_df.index + 1
# assign RUL to a newly-named column, max
truth_df['max'] = rul['max'] + truth_df['rul_init']
truth_df.drop('rul_init', axis=1, inplace=True)

# Merge Truth Dataframe with Test Dataframe to put the actual RUL together with the data in Test Dataframe
# Merge based on Engine ID
test_df = test_df.merge(truth_df, on=['id'], how='left')
# Final RUL matched to Engine ID
test_df['RUL'] = test_df['max'] - test_df['cycle']

# Clipping the maximum RUL to aircraft engines.
test_df['RUL'] = test_df['RUL'].clip(upper=125)
test_df.drop('max', axis=1, inplace=True)

* 2g: Transform data into a form that can be fed into the Deep Learning Model as input.
> [samples, time steps, features]

In [152]:
# Transforming data into a form that can be fed in to the Deep Learning Model 
# The Form is a(3 Dimensional Form): (samples, time steps, features) 

# Assign sequence length of 50 
sequence_length = 50
# Generate the sequence  
def gen_sequence(id_df, seq_length, seq_cols):
    data_matrix = id_df[seq_cols].values
    num_elements = data_matrix.shape[0]
    for start, stop in zip(range(0, num_elements-seq_length), range(seq_length, num_elements)):
        yield data_matrix[start:stop, :]

In [153]:
# Feature columns consists of the Operational Settings and Sensor Columns 
sensor_cols = ['s' + str(i) for i in range(1,22)]
sequence_cols = ['setting1', 'setting2', 'setting3', 'cycle_norm']
sequence_cols.extend(sensor_cols)

seq_gen = (list(gen_sequence(train_df[train_df['id']==id], sequence_length, sequence_cols)) 
           for id in train_df['id'].unique())

# Generate a sequence with the gen_sequence function to get the 3-Dimensional Form 
# Convert to numpy array
seq_array = np.concatenate(list(seq_gen)).astype(np.float32)

In [154]:
# Generate the RUL labels 
def gen_labels(id_df, seq_length, label):
    data_matrix = id_df[label].values
    num_elements = data_matrix.shape[0]
    return data_matrix[seq_length:num_elements, :]

# Generate the labels using the gen_labels function
label_gen = [gen_labels(train_df[train_df['id']==id], sequence_length, ['RUL']) 
             for id in train_df['id'].unique()]
# Convert to numpy array
label_array = np.concatenate(label_gen).astype(np.float32)
label_array.shape

(15631, 1)

## 3. Model Building

* 3a. Defining R2 to be used as an evaluation metric
> R2 is a statistical measures which gives an indication of closeness of fit.

In [155]:
# Defining R2 to be used as an evaluation metric in the Deep Learning Model Evaluation Metrics
def r2_keras(y_true, y_pred):
    SS_res =  K.sum(K.square( y_true - y_pred ))
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) )
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

* 3b. Defining Adjusted R2 to be used as an evaluation metric
> The adjusted R-squared is a modified version of R-squared that accounts for predictors that are not significant in a regression model.

In [ ]:
# Adjusted R2 score
def adjusted_r2(r2, p, n):
  numerator = (1-r2)*(n-1)
  denom = n - p - 1
  result = 1 - (numerator/denom)
  return result

print('Adjusted R2:')
print(adjusted_r2(scores[2], nb_features, len(label_array)))



## Deep Learning Architecture and Topology

* 3c. Deep Learning Architecture consists of:
<br> > Sequential Model
<br> > Bi-LSTM Layer
<br> > LSTM Layer
<br> > Dropout Layers

In [156]:
# Building the Deep Learning Architecture

# Defining the features to be fed into the input of the layers in the Deep Learning Model
nb_features = seq_array.shape[2]
nb_out = label_array.shape[1]

# Sequential Model
model = Sequential()
# Add a Bi-LSTM Layer
model.add(Bidirectional(LSTM(20, return_sequences=True), input_shape=(sequence_length, nb_features)))
# Add a Dropout Layer after the Bi-LSTM Layer to minimize overfitting  
model.add(Dropout(0.2))
# Add a LSTM Layer
model.add(LSTM(units=50,return_sequences=False, activation = 'tanh'))
# Add a Dropout Layer after the LSTM Layer to minimize overfitting  
model.add(Dropout(0.2))
# Dense Layer
model.add(Dense(units=nb_out))
# Activation Function 
model.add(Activation("linear"))
# Compile model, set metrics for evaluation
model.compile(loss='mean_squared_error', optimizer='rmsprop',metrics=['mse',r2_keras])
# For model observation
print(model.summary())

# Fit the Deep Learning network on our training data
history = model.fit(seq_array, label_array, epochs=5, batch_size=10, validation_split=0.05, verbose=2,
          callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=0, mode='min'),
                       keras.callbacks.ModelCheckpoint(model_path,monitor='val_loss', save_best_only=True, mode='min', verbose=0)])


Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_9 (Bidirection (None, 50, 40)            7360      
_________________________________________________________________
dropout_22 (Dropout)         (None, 50, 40)            0         
_________________________________________________________________
lstm_18 (LSTM)               (None, 50)                18200     
_________________________________________________________________
dropout_23 (Dropout)         (None, 50)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 51        
_________________________________________________________________
activation_8 (Activation)    (None, 1)                 0         
Total params: 25,611
Trainable params: 25,611
Non-trainable params: 0
__________________________________________________

* 3d. Training Data Metrics

In [157]:
# Metrics from fitting the model on our Training Data
scores = model.evaluate(seq_array, label_array, verbose=1, batch_size=200)
print('\nMSE: {}'.format(scores[1]))
print('\nR^2: {}'.format(scores[2]))

79/79 [==============================] - 0s 5ms/step - loss: 166.2797 - mse: 166.2797 - r2_keras: 0.8655

MSE: 166.27967834472656

R^2: 0.8654634356498718


In [158]:
# Adjusted R2 score
def adjusted_r2(r2, p, n):
  numerator = (1-r2)*(n-1)
  denom = n - p - 1
  result = 1 - (numerator/denom)
  return result

print('Adjusted R2:')
print(adjusted_r2(scores[2], nb_features, len(label_array)))



Adjusted R2:
0.8652479012628963


* 3e. Preparation of Test Data to feed into our model for evaluation

In [159]:
# Preparing Test Data to be fed into our model for evaluation
seq_array_test_last = [test_df[test_df['id']==id][sequence_cols].values[-sequence_length:] 
                       for id in test_df['id'].unique() if len(test_df[test_df['id']==id]) >= sequence_length]

seq_array_test_last = np.asarray(seq_array_test_last).astype(np.float32)

In [160]:
# Preparing Test Data to be fed into our model for evaluation
y_mask = [len(test_df[test_df['id']==id]) >= sequence_length for id in test_df['id'].unique()]
label_array_test_last = test_df.groupby('id')['RUL'].nth(-1)[y_mask].values
label_array_test_last = label_array_test_last.reshape(label_array_test_last.shape[0],1).astype(np.float32)

* 3f. Testing Data Metrics

In [161]:
# Test Data Metrics after running our model
scores_test = model.evaluate(seq_array_test_last, label_array_test_last, verbose = 1, batch_size = 200)
print('\nMSE: {}'.format(scores_test[1]))
print('\nR^2: {}'.format(scores_test[2]))
print('\nRMSE:')
print(math.sqrt(scores_test[1]))

1/1 [==============================] - 0s 18ms/step - loss: 176.8562 - mse: 176.8562 - r2_keras: 0.8892

MSE: 176.8561553955078

R^2: 0.889172375202179

RMSE:
13.298727585581554


# Predicted Values

In [162]:
# To obtain our model's prediction on the Test Data
scores_test = model.predict(seq_array_test_last)
scores_test


array([[ 67.94558  ],
       [ 95.82517  ],
       [102.17069  ],
       [113.70323  ],
       [111.678925 ],
       [101.63217  ],
       [115.98318  ],
       [ 80.206314 ],
       [101.06471  ],
       [ 97.11608  ],
       [ 90.80011  ],
       [102.6706   ],
       [ 94.159325 ],
       [ 58.36712  ],
       [ 31.211872 ],
       [ 87.43528  ],
       [  7.7870865],
       [ 77.095566 ],
       [113.072586 ],
       [ 18.947838 ],
       [116.63224  ],
       [ 70.306786 ],
       [113.351494 ],
       [ 96.63779  ],
       [ 94.49725  ],
       [  3.8548162],
       [ 46.69287  ],
       [107.671646 ],
       [  3.5280566],
       [  2.298322 ],
       [ 10.054273 ],
       [ 28.816008 ],
       [ 54.738876 ],
       [ 18.172281 ],
       [ 23.952404 ],
       [  8.486936 ],
       [ 78.33343  ],
       [117.72732  ],
       [ 87.320885 ],
       [ 43.000294 ],
       [113.288895 ],
       [114.36866  ],
       [ 19.53542  ],
       [ 77.72872  ],
       [107.537575 ],
       [ 3